In [ ]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
import numpy as np
import os
from tqdm import trange
import pandas as pd

from utils import plot_me, smooth_traces, generate_track_interactions
from track import Track

In [4]:
##########################################################################
####################### HELPER FUNCTIONS #################################
##########################################################################

#
def parse_spreadsheet_return_unique_vids_and_starts_ends(header, 
                                                         dataframe):

    # get animal ids first
    animal_labels = []
    for k in range(1,len(header),2):
        temp = header[k]
        animal_labels.append(temp)

    #print ("animal_labels: ", animal_labels)

    # get starts and ends next
    starts_ends = []
    video_names = []
    animal_ids = []
    data_npy = dataframe.to_numpy()
    #print (data_npy.shape)

    #
    for k in range(data_npy.shape[0]):
        vid_name = data_npy[k][0]
        for p in range(1,data_npy.shape[1],2):
            #print (data_npy[k,p])
            if data_npy[k,p]!=0:
                starts_ends.append([data_npy[k,p],data_npy[k,p+1]])
                video_names.append(vid_name)
                animal_ids.append(animal_labels[p//2])

    #
    video_names = np.array(video_names)
    starts_ends = np.array(starts_ends)
    animal_ids = np.array(animal_ids)

    # #
    # for k in range(len(starts_ends)):
    #     print (video_names[k], 
    #            starts_ends[k],
    #            animal_ids[k])

    # gather all video-animal pairs that are the same 
    unique_vids = np.unique(video_names)
    unique_ids = np.unique(animal_ids)
    #print (unique_ids)

    # 
    ctr=0
    final_arrays= []
    final_arrays.append([])
    for unique_vid in unique_vids:
        for unique_id in unique_ids:
           
            
            # loop over all vids
            se_array = []
            for k in range(starts_ends.shape[0]):
                if video_names[k] == unique_vid:
                    if animal_ids[k] == unique_id:
                        se_array.append(starts_ends[k])
            #
            if len(se_array)>0:
                final_arrays[ctr].append(unique_vid)
                final_arrays[ctr].append(unique_id)
                final_arrays[ctr].append(np.vstack(se_array).squeeze())
                final_arrays.append([])
                ctr+=1
    
    if len(final_arrays[-1])==0:
        del(final_arrays[-1])

    # 
    print ("                                UNIQUE VIDEO ID           ANIMAL IDS      STARTS-ENDS ARRAYS")
    for k in range(len(final_arrays)):
        print (" unique video - pair: #", k, ": ", final_arrays[k])

    return final_arrays

#
def make_pairwise_npy_files_for_simba(final_arrays,
                                     root_dir):
    #
    for dataset in final_arrays:

        #
        fname = dataset[0]

        #
        animal_ids = np.int32([dataset[1][0],dataset[1][2]])

        #
        starts_ends = dataset[2]

        fname_out = os.path.join(root_dir+'/pairwise'+
                            fname+"_animals_"+str(animal_ids[0])+"_"+str(animal_ids[1])+'.npy')
        
        #
        if os.path.exists(fname_out):
            continue
        
        #
        print ("PROCESSING: ")
        print (fname)
        print (animal_ids)
        print (starts_ends)
        
        #
        track = Track(os.path.join(root_dir, 
                                   fname+'_compressed_Day.slp'))
        track.animal_ids = animal_ids
        track.tracks_names = ['female','male','pup1','pup2','pup3','pup4']
        track.recompute_spine_centres=True
        track.verbose = True                         # gives additional printouts
        track.n_animals = len(track.animal_ids)      # number of animals
        track.filter_width = 10                      # this is the median filter width in frames; e.g. 10 ~=0.4 seconds
                                                     # higher values provide more stability, but less temporally precise locations
        # 
        track.load_tracks()

        #
        data_out = np.zeros((track.tracks_spine.shape[0],
                             2,2,2))

        #
        data_out[:,:,:,0] = track.tracks_spine[:,animal_ids]

        #
        try:
            for se in starts_ends:
                data_out[int(se[0]):int(se[1]),:,:,1]=1
        except:
            print ("single exmaple only")
            data_out[int(starts_ends[0]):int(starts_ends[1]),:,:,1]=1

        #
        np.save(fname_out, data_out)
        
        ####################################################
        ############# MAKE ANIMAL ANGLES ###################
        ####################################################
        
        track.smooth_angles = True
        track.compute_angles_from_features(fname_out)
        np.save(fname_out[:-4]+"_angles.npy",
                track.angles)    
        
        #
        print ("")


In [5]:

#
def make_pairwise_npy_files_for_simba_single_file(fname_in,
                                                   root_dir):
   

    dataset = 

        #
        fname = dataset[0]

        #
        animal_ids = np.int32([dataset[1][0],dataset[1][2]])

        #
        starts_ends = dataset[2]

        fname_out = os.path.join(root_dir+'/pairwise'+
                            fname+"_animals_"+str(animal_ids[0])+"_"+str(animal_ids[1])+'.npy')
        
        #
        if os.path.exists(fname_out):
            continue
        
        #
        print ("PROCESSING: ")
        print (fname)
        print (animal_ids)
        print (starts_ends)
        
        #
        track = Track(os.path.join(root_dir, 
                                   fname+'_compressed_Day.slp'))
        track.animal_ids = animal_ids
        track.tracks_names = ['female','male','pup1','pup2','pup3','pup4']
        track.recompute_spine_centres=True
        track.verbose = True                         # gives additional printouts
        track.n_animals = len(track.animal_ids)      # number of animals
        track.filter_width = 10                      # this is the median filter width in frames; e.g. 10 ~=0.4 seconds
                                                     # higher values provide more stability, but less temporally precise locations
        # 
        track.load_tracks()

        #
        data_out = np.zeros((track.tracks_spine.shape[0],
                             2,2,2))

        #
        data_out[:,:,:,0] = track.tracks_spine[:,animal_ids]

        #
        try:
            for se in starts_ends:
                data_out[int(se[0]):int(se[1]),:,:,1]=1
        except:
            print ("single exmaple only")
            data_out[int(starts_ends[0]):int(starts_ends[1]),:,:,1]=1

        #
        np.save(fname_out, data_out)
        
        ####################################################
        ############# MAKE ANIMAL ANGLES ###################
        ####################################################
        
        track.smooth_angles = True
        track.compute_angles_from_features(fname_out)
        np.save(fname_out[:-4]+"_angles.npy",
                track.angles)    
        
        #
        print ("")
        
        
#
fname = '/home/cat/Downloads/approach/2020_07_28_17_11_05_340439_compressed_Day_0_1_cleaned.npy'

#
make_pairwise_npy_files_for_simba_single_file

SyntaxError: invalid syntax (4024706071.py, line 6)

In [10]:
##############################################################
################# LOAD DATA FRAME AND ID PAIRS ###############
##############################################################

# these are the ids of the gerbils that Lisa and Jennifer decided to label
# ids 0->5 correspond to female->pup4
id_pairs = [
    [0,1],
    [1,0],
    [4,5],
    [5,4],
    [0,2],
    [2,0],
    [1,3],
    [3,1]
]

fname = "/home/cat/Downloads/chasingvf.csv"

#
dataframe = pd.read_csv(fname)
header = dataframe.columns.tolist()

print (header)
dataframe



['video_name', '0,1', '0,1.1', '1,0', '1,0.1', '4,5', '4,5.1', '5,4', '5,4.1', '0,2', '0,2.1', '2,0', '2,0.1', '1,3', '1,3.1', '3,1', '3,1.1']


,video_name,"0,1","0,1.1","1,0","1,0.1","4,5","4,5.1","5,4","5,4.1","0,2","0,2.1","2,0","2,0.1","1,3","1,3.1","3,1","3,1.1"
0,2020_08_02_15_50_24_856237,1000,1022,0,0,13271,13294,0,0,0,0,0,0,0,0,0,0
1,2020_08_02_15_50_24_856237,1154,1167,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2020_08_02_15_50_24_856237,11044,11081,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2020_08_01_11_02_57_239373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2020_07_31_11_22_34_209876,28663,28706,0,0,20594,20612,0,0,0,0,0,0,0,0,21720,21732
5,2020_07_31_11_22_34_209876,0,0,0,0,20618,20635,0,0,0,0,0,0,0,0,0,0
6,2020_07_29_11_58_37_808577,22746,22760,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2020_07_27_10_07_16_505127,0,0,20830,20843,0,0,0,0,0,0,0,0,0,0,0,0
8,2020_07_25_12_15_56_892610,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,2020_07_24_14_49_21_129654,15664,15678,0,0,22160,22168,0,0,0,0,15035,15052,0,0,0,0


In [11]:
#########################################################################
################# CONVERT SPREADSHEET TO 2 COLUMN VERSION ###############
#########################################################################
final_arrays = parse_spreadsheet_return_unique_vids_and_starts_ends(header, dataframe)
            

                                UNIQUE VIDEO ID           ANIMAL IDS      STARTS-ENDS ARRAYS
 unique video - pair: # 0 :  ['2020_07_21_11_48_16_485044', '0,1', array([5966, 5987])]
 unique video - pair: # 1 :  ['2020_07_22_11_48_50_324539', '4,5', array([9614, 9642])]
 unique video - pair: # 2 :  ['2020_07_22_17_21_14_939501', '1,0', array([12510, 12527])]
 unique video - pair: # 3 :  ['2020_07_24_14_49_21_129654', '0,1', array([15664, 15678])]
 unique video - pair: # 4 :  ['2020_07_24_14_49_21_129654', '0,2', array([16416, 16435])]
 unique video - pair: # 5 :  ['2020_07_24_14_49_21_129654', '2,0', array([[15035, 15052],
       [15154, 15167]])]
 unique video - pair: # 6 :  ['2020_07_24_14_49_21_129654', '4,5', array([22160, 22168])]
 unique video - pair: # 7 :  ['2020_07_24_15_11_26_425064', '0,1', array([22724, 22738])]
 unique video - pair: # 8 :  ['2020_07_24_15_11_26_425064', '1,0', array([13792, 13813])]
 unique video - pair: # 9 :  ['2020_07_24_18_08_37_599524', '0,1', array([[ 

In [12]:
###########################################################################################
########## GENERATE .NPY FILES FROM .SLP FOR EACH UNIQUE VID-PAIR AND NUMPY ARRAYS ########
###########################################################################################

##############################################################
root_dir = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/features'

# break
make_pairwise_npy_files_for_simba(final_arrays,
                                     root_dir)  
print ("DONE...")

PROCESSING: 
2020_07_21_11_48_16_485044
[0 1]
[5966 5987]
single exmaple only
(28802, 2, 2, 2)


100%|███████████████████████████████████████████████████████████████| 28802/28802 [00:02<00:00, 13045.73it/s]



PROCESSING: 
2020_07_22_11_48_50_324539
[4 5]
[9614 9642]
single exmaple only
(28802, 2, 2, 2)


100%|███████████████████████████████████████████████████████████████| 28802/28802 [00:01<00:00, 14414.45it/s]



PROCESSING: 
2020_07_22_17_21_14_939501
[1 0]
[12510 12527]
single exmaple only
(28802, 2, 2, 2)


...


PROCESSING: 
2020_08_02_15_50_24_856237
[4 5]
[13271 13294]
single exmaple only
(28802, 2, 2, 2)


100%|███████████████████████████████████████████████████████████████| 28802/28802 [00:02<00:00, 13427.71it/s]



PROCESSING: 
2020_08_03_16_59_52_513117
[0 1]
[18084 18098]
single exmaple only
(28802, 2, 2, 2)


100%|███████████████████████████████████████████████████████████████| 28802/28802 [00:02<00:00, 13053.99it/s]


DONE...


In [13]:
data = np.load('/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/simba/2020_08_01_11_02_57_239373/2020_08_01_11_02_57_239373_animals_5_4.npy')

print ("# frames, # animals, x-y, locations and behavior yes/no")
print (data.shape)

frame_id = 20280  #<--- example of on behavior
animal_id = 0
print ("frame: ", frame_id, "animal id: ", animal_id, ": locations: ", data[frame_id,animal_id,:,0])
print ("frame: ", frame_id, "animal id: ", animal_id, ": behavior state: ", data[frame_id,animal_id,:,1])

# frames, # animals, x-y, locations and behavior yes/no
(28802, 2, 2, 2)
frame:  20280 animal id:  0 : locations:  [544.71557617 563.74786377]
frame:  20280 animal id:  0 : behavior state:  [1. 1.]
